In [86]:
import os

# Define the base path where plots folder exists
base_path = r"D:\PythonBOOTCAMP\Machine Learning Project\plots"

# Create the 'resistance_plots' and 'impedance_plots' folders if they don't exist
resistance_plots_path = os.path.join(base_path, "resistance_plots")
impedance_plots_path = os.path.join(base_path, "impedance_plots")

# Check if the directories exist, if not create them
if not os.path.exists(resistance_plots_path):
    os.makedirs(resistance_plots_path)

if not os.path.exists(impedance_plots_path):
    os.makedirs(impedance_plots_path)

print(f"Created 'resistance_plots' at: {resistance_plots_path}")
print(f"Created 'impedance_plots' at: {impedance_plots_path}")


Created 'resistance_plots' at: D:\PythonBOOTCAMP\Machine Learning Project\plots\resistance_plots
Created 'impedance_plots' at: D:\PythonBOOTCAMP\Machine Learning Project\plots\impedance_plots


In [87]:
import pandas as pd
import ast
from datetime import datetime
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import kaleido

# Set renderer to work in Kaggle notebooks
pio.renderers.default = 'iframe_connected'

# Load data from the correct path
metadata_df = pd.read_csv("D:/PythonBOOTCAMP/Machine Learning Project/data/metadata.csv")  # Corrected path

# Filter for impedance rows and select necessary columns
impedance_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'Re', 'Rct', 'battery_id']]
file_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'filename', 'battery_id']]

# Function to parse start_time
def parse_start_time(value):
    try:
        if isinstance(value, str):
            value = value.strip("[]").replace(",", "")  # Remove brackets and commas
            components = [float(x) for x in value.split()]  # Split and convert to float
            if len(components) == 6:
                year, month, day, hour, minute = map(int, components[:5])
                second = int(components[5])  # Handle fractional seconds
                return datetime(year, month, day, hour, minute, second)
        elif isinstance(value, (list, np.ndarray)) and len(value) == 6:
            year, month, day, hour, minute = map(int, value[:5])
            second = int(float(value[5]))  # Handle fractional seconds
            return datetime(year, month, day, hour, minute, second)
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Failed to parse: {value}, Error: {e}")
    return pd.NaT

def parse_Re_or_Rct(value):
    return abs(complex(value))

# Apply parsing function
impedance_df['start_time'] = impedance_df['start_time'].apply(parse_start_time)
file_df['start_time'] = file_df['start_time'].apply(parse_start_time)
impedance_df['Re'] = impedance_df['Re'].apply(parse_Re_or_Rct)
impedance_df['Rct'] = impedance_df['Rct'].apply(parse_Re_or_Rct)


In [88]:
from PIL import Image
import img2pdf
import os

# Specify the directory containing the images
directory_path = "D:/PythonBOOTCAMP/Machine Learning Project/plots/impedance_plots"

# Check if the directory exists
if not os.path.exists(directory_path):
    print(f"The directory {directory_path} does not exist.")
else:
    # Create a list of image file paths
    image_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Check if there are any images in the directory
    if not image_files:
        print(f"No image files found in {directory_path}.")
    else:
        try:
            # Process images and fill transparency with white background
            filled_images = []
            for img_file in image_files:
                with Image.open(img_file) as img:
                    # If the image has an alpha channel (transparency), fill it with a white background
                    if img.mode == 'RGBA':
                        img = img.convert('RGBA')
                        background = Image.new('RGBA', img.size, (255, 255, 255, 255))  # White background
                        background.paste(img, (0, 0), img)
                        img = background.convert('RGB')  # Convert back to RGB
                    # Save the modified image temporarily
                    temp_path = img_file + '.filled.jpg'
                    img.save(temp_path, 'JPEG')
                    filled_images.append(temp_path)
            
            # Convert images to PDF
            with open("impedance_plots.pdf", "wb") as file:
                file.write(img2pdf.convert(filled_images))

            print("PDF successfully created with the images.")
            
            # Clean up the temporary files
            for temp_img in filled_images:
                os.remove(temp_img)

        except Exception as e:
            print(f"An error occurred during the conversion: {e}")


PDF successfully created with the images.


In [89]:
import img2pdf
import os
from PIL import Image

# Specify the directory containing images
directory_path = "D:/PythonBOOTCAMP/Machine Learning Project/plots/resistance_plots"

# Create a list of image file paths
image_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Create a list to store processed images (converted to RGB if necessary)
processed_images = []

# Iterate over images and convert them if they have an alpha channel (transparency)
for image_file in image_files:
    with Image.open(image_file) as img:
        # Convert image to 'RGB' if it has an alpha channel (transparency)
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        processed_images.append(img)

# Save the processed images to a temporary location before creating the PDF
temp_image_files = []
for idx, img in enumerate(processed_images):
    temp_path = f"temp_image_{idx}.jpg"
    img.save(temp_path, format='JPEG')
    temp_image_files.append(temp_path)

# Convert images to PDF
with open("resistance_plots.pdf", "wb") as file:
    file.write(img2pdf.convert(temp_image_files))

# Optionally, delete the temporary images after creating the PDF
for temp_file in temp_image_files:
    os.remove(temp_file)

print("PDF successfully created with the images.")


PDF successfully created with the images.


In [1]:
import pandas as pd
import plotly.graph_objects as go
import os
from datetime import datetime

# Step 1: Load and Prepare Data
# Load metadata CSV
metadata_path = "D:/PythonBOOTCAMP/Machine Learning Project/data/metadata.csv"
metadata_df = pd.read_csv(metadata_path)

# Filter impedance data and select required columns
impedance_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'Re', 'Rct']]

# Parse 'start_time' column to datetime format
def parse_start_time(value):
    try:
        if isinstance(value, str):
            components = [float(x) for x in value.strip("[]").replace(",", "").split()]
            if len(components) == 6:
                year, month, day, hour, minute = map(int, components[:5])
                second = int(components[5])
                return datetime(year, month, day, hour, minute, second)
    except Exception as e:
        print(f"Error parsing time: {value}, {e}")
    return pd.NaT

impedance_df['start_time'] = impedance_df['start_time'].apply(parse_start_time)

# Drop rows with invalid dates
impedance_df = impedance_df.dropna()

# Step 2: Create Folders for Plots
resistance_plots_folder = "D:/PythonBOOTCAMP/Machine Learning Project/plots/resistance_plots"
impedance_plots_folder = "D:/PythonBOOTCAMP/Machine Learning Project/plots/impedance_plots"

os.makedirs(resistance_plots_folder, exist_ok=True)
os.makedirs(impedance_plots_folder, exist_ok=True)

# Step 3: Plot Re (Electrolyte Resistance) Over Time
fig_re = go.Figure()
fig_re.add_trace(go.Scatter(
    x=impedance_df['start_time'], 
    y=impedance_df['Re'], 
    mode='lines+markers',
    name='Electrolyte Resistance (Re)',
    line=dict(color='blue')
))

fig_re.update_layout(
    title="Electrolyte Resistance (Re) Over Time",
    xaxis_title="Time (Start Time)",
    yaxis_title="Re (Ohms)",
    template="plotly_dark"
)

# Save Re plot
re_plot_path = os.path.join(resistance_plots_folder, "re_over_time.png")
fig_re.write_image(re_plot_path)
print(f"Re plot saved at: {re_plot_path}")

# Step 4: Plot Rct (Charge Transfer Resistance) Over Time
fig_rct = go.Figure()
fig_rct.add_trace(go.Scatter(
    x=impedance_df['start_time'], 
    y=impedance_df['Rct'], 
    mode='lines+markers',
    name='Charge Transfer Resistance (Rct)',
    line=dict(color='orange')
))

fig_rct.update_layout(
    title="Charge Transfer Resistance (Rct) Over Time",
    xaxis_title="Time (Start Time)",
    yaxis_title="Rct (Ohms)",
    template="plotly_dark"
)

# Save Rct plot
rct_plot_path = os.path.join(impedance_plots_folder, "rct_over_time.png")
fig_rct.write_image(rct_plot_path)
print(f"Rct plot saved at: {rct_plot_path}")

# Step 5: Combined Plot for Re and Rct
fig_combined = go.Figure()

fig_combined.add_trace(go.Scatter(
    x=impedance_df['start_time'],
    y=impedance_df['Re'],
    mode='lines+markers',
    name='Re (Ohms)',
    line=dict(color='blue')
))

fig_combined.add_trace(go.Scatter(
    x=impedance_df['start_time'],
    y=impedance_df['Rct'],
    mode='lines+markers',
    name='Rct (Ohms)',
    line=dict(color='orange')
))

fig_combined.update_layout(
    title="Re and Rct Over Time",
    xaxis_title="Time (Start Time)",
    yaxis_title="Resistance (Ohms)",
    template="plotly_dark"
)

# Save combined plot
combined_plot_path = "D:/PythonBOOTCAMP/Machine Learning Project/plots/combined_resistance_plot.png"
fig_combined.write_image(combined_plot_path)
print(f"Combined plot saved at: {combined_plot_path}")


Re plot saved at: D:/PythonBOOTCAMP/Machine Learning Project/plots/resistance_plots\re_over_time.png
Rct plot saved at: D:/PythonBOOTCAMP/Machine Learning Project/plots/impedance_plots\rct_over_time.png
Combined plot saved at: D:/PythonBOOTCAMP/Machine Learning Project/plots/combined_resistance_plot.png


In [2]:
import img2pdf
import os
from PIL import Image

# Paths to the plots
resistance_plot_path = "D:/PythonBOOTCAMP/Machine Learning Project/plots/resistance_plots/re_over_time.png"
impedance_plot_path = "D:/PythonBOOTCAMP/Machine Learning Project/plots/impedance_plots/rct_over_time.png"
combined_plot_path = "D:/PythonBOOTCAMP/Machine Learning Project/plots/combined_resistance_plot.png"

# List of image files
image_files = [resistance_plot_path, impedance_plot_path, combined_plot_path]

# Convert images to PDF
with open("battery_plots.pdf", "wb") as file:
    file.write(img2pdf.convert(image_files))

print("PDF successfully created with the plots.")


Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.


PDF successfully created with the plots.


In [3]:
import img2pdf
import os
from PIL import Image

# Specify the directory containing images
directory_path = "D:/PythonBOOTCAMP/Machine Learning Project/plots/resistance_plots"

# Create a list of image file paths
image_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Create a list to store processed images (converted to RGB if necessary)
processed_images = []

# Iterate over images and convert them to 'RGB' (removing transparency)
for image_file in image_files:
    with Image.open(image_file) as img:
        # Convert image to 'RGB' if it has an alpha channel (transparency)
        if img.mode == 'RGBA' or img.mode == 'LA':
            img = img.convert('RGB')
        processed_images.append(img)

# Save the processed images to a temporary location before creating the PDF
temp_image_files = []
for idx, img in enumerate(processed_images):
    temp_path = f"temp_image_{idx}.jpg"
    img.save(temp_path, format='JPEG')
    temp_image_files.append(temp_path)

# Convert images to PDF
with open("resistance_plots.pdf", "wb") as file:
    file.write(img2pdf.convert(temp_image_files))

# Optionally, delete the temporary images after creating the PDF
for temp_file in temp_image_files:
    os.remove(temp_file)

print("PDF successfully created with the images.")


PDF successfully created with the images.


In [4]:
import plotly.graph_objects as go

# Create a combined plot showing both Re and Rct over time
fig_combined = go.Figure()

fig_combined.add_trace(go.Scatter(x=impedance_df['start_time'], y=impedance_df['Re'], mode='lines', name='Re (Electrolyte Resistance)'))
fig_combined.add_trace(go.Scatter(x=impedance_df['start_time'], y=impedance_df['Rct'], mode='lines', name='Rct (Charge Transfer Resistance)'))

# Update layout to improve readability
fig_combined.update_layout(
    title="Re and Rct over Time",
    xaxis_title="Time",
    yaxis_title="Resistance (Ohms)",
    legend_title="Parameters",
    template="plotly_dark"
)

# Save the combined plot to a PNG file
combined_plot_path = r"D:\PythonBOOTCAMP\Machine Learning Project\plots\combined_resistance_plot.png"
fig_combined.write_image(combined_plot_path)

# Output file location message
print(f"Combined plot saved at: {combined_plot_path}")


Combined plot saved at: D:\PythonBOOTCAMP\Machine Learning Project\plots\combined_resistance_plot.png


In [6]:
from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Add a page
pdf.add_page()

# Set title
pdf.set_font("Arial", size=16, style='B')
pdf.cell(200, 10, txt="Battery Aging Analysis - Re and Rct", ln=True, align='C')

# Add analysis text
pdf.ln(10)
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 10, txt="In this report, we analyze how the electrolyte resistance (Re) and charge transfer resistance (Rct) of the battery change over time as the battery ages. "
                           "The following plots show the resistance parameters over the battery's life cycle, indicating how the battery's internal parameters evolve as it goes through charge/discharge cycles.")

# Add plot images to the PDF
pdf.ln(10)
pdf.image(r"D:\PythonBOOTCAMP\Machine Learning Project\plots\combined_resistance_plot.png", x=10, y=50, w=180)

# Save the PDF
pdf_output_path = r"D:\PythonBOOTCAMP\Machine Learning Project\battery_analysis_report.pdf"
pdf.output(pdf_output_path)

print(f"PDF report generated and saved at: {pdf_output_path}")


PDF report generated and saved at: D:\PythonBOOTCAMP\Machine Learning Project\battery_analysis_report.pdf


In [7]:
from fpdf import FPDF
import os

# Create instance of FPDF class
pdf = FPDF()

# Add a page to the PDF
pdf.add_page()

# Set title and font for the report
pdf.set_font("Arial", 'B', 16)
pdf.cell(200, 10, "Battery Aging and Impedance Analysis", ln=True, align="C")

# Add description
pdf.ln(10)  # Line break
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 10, """
This report presents an analysis of the aging process of Li-ion batteries, based on the NASA Battery Dataset.
The analysis includes battery impedance (Re and Rct) measurements, which were taken over several charge/discharge cycles.
The plots below show how the battery parameters change as the battery ages.
The experiments were stopped when the batteries reached their end-of-life (EOL) criteria.
""")

# Add Re plot
pdf.ln(10)  # Line break
pdf.cell(200, 10, "Plot 1: Re (Electrolyte Resistance) over Time", ln=True)
pdf.image("D:/PythonBOOTCAMP/Machine Learning Project/plots/resistance_plots/re_over_time.png", x=10, w=180)

# Add Rct plot
pdf.ln(120)  # Line break
pdf.cell(200, 10, "Plot 2: Rct (Charge Transfer Resistance) over Time", ln=True)
pdf.image("D:/PythonBOOTCAMP/Machine Learning Project/plots/impedance_plots/rct_over_time.png", x=10, w=180)

# Add Combined Plot
pdf.ln(120)  # Line break
pdf.cell(200, 10, "Plot 3: Combined Re and Rct over Time", ln=True)
pdf.image("D:/PythonBOOTCAMP/Machine Learning Project/plots/combined_resistance_plot.png", x=10, w=180)

# Save the PDF report
pdf_output_path = "D:/PythonBOOTCAMP/Machine Learning Project/battery_analysis_report.pdf"
pdf.output(pdf_output_path)

print(f"PDF report generated and saved at: {pdf_output_path}")


PDF report generated and saved at: D:/PythonBOOTCAMP/Machine Learning Project/battery_analysis_report.pdf


In [8]:
from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Add a page to the PDF
pdf.add_page()

# Set font for title
pdf.set_font("Arial", 'B', 16)
pdf.cell(200, 10, "Battery Aging and Impedance Analysis", ln=True, align='C')

# Set font for body text
pdf.set_font("Arial", '', 12)

# Introduction
pdf.ln(10)
pdf.multi_cell(0, 10, 
    """This report presents an analysis of the aging process of Li-ion batteries, based on the NASA Battery
Dataset. The analysis includes battery impedance (Re and Rct) measurements, which were taken over
several charge/discharge cycles. The experiments were stopped when the batteries reached their end-of-life
(EOL) criteria. This report aims to explore how these battery parameters evolve over time during the aging
process of the batteries.""")

# Methodology
pdf.ln(10)
pdf.multi_cell(0, 10, 
    """Methodology:
    The dataset was filtered for impedance data and the start time, Re, Rct, and battery ID were extracted for analysis.
    The impedance measurements (Re and Rct) were plotted over time to observe how these parameters change as the 
    battery ages. The plots represent the electrolyte resistance (Re) and charge transfer resistance (Rct), which are 
    crucial for understanding the internal condition of the battery during operation.""")

# Plot 1: Re (Electrolyte Resistance) over Time
pdf.ln(10)
pdf.multi_cell(0, 10, "Plot 1: Re (Electrolyte Resistance) over Time")
# Add the image of the Re plot
pdf.image("D:/PythonBOOTCAMP/Machine Learning Project/plots/resistance_plots/re_over_time.png", x=10, w=180)

# Plot 2: Rct (Charge Transfer Resistance) over Time
pdf.ln(10)
pdf.multi_cell(0, 10, "Plot 2: Rct (Charge Transfer Resistance) over Time")
# Add the image of the Rct plot
pdf.image("D:/PythonBOOTCAMP/Machine Learning Project/plots/impedance_plots/rct_over_time.png", x=10, w=180)

# Analysis of Results
pdf.ln(10)
pdf.multi_cell(0, 10, 
    """Analysis of Results:
    - Re (Electrolyte Resistance) over Time: The plot demonstrates how the resistance of the electrolyte increases over 
    time, which could indicate the internal degradation of the battery material as it ages.
    - Rct (Charge Transfer Resistance) over Time: The plot illustrates how the charge transfer resistance changes, which 
    provides insight into how effectively the battery can transfer charge, an important indicator of battery performance.""")

# Conclusion
pdf.ln(10)
pdf.multi_cell(0, 10, 
    """Conclusion:
    The analysis of Re and Rct indicates that as the battery ages, both the electrolyte resistance and charge transfer 
    resistance increase, which signifies degradation of the battery. These trends are consistent with the expected behavior 
    of Li-ion batteries as they undergo charge/discharge cycles. Future studies could investigate additional parameters 
    affecting battery life and performance, and explore methods for improving battery longevity.""")

# Save the PDF report
pdf_output_path = "D:/PythonBOOTCAMP/Machine Learning Project/battery_analysis_report_final.pdf"
pdf.output(pdf_output_path)

print(f"Updated PDF report generated and saved at: {pdf_output_path}")


Updated PDF report generated and saved at: D:/PythonBOOTCAMP/Machine Learning Project/battery_analysis_report_final.pdf
